### First use adb and scrpy to screen share your device

In [1]:
import time, subprocess, ctypes
import numpy as np
import cv2

## Functions

In [2]:
def get_window_geometry_from_wmctrl(target_title):
    """
    Uses wmctrl -lG to find a window whose title contains target_title.
    Returns (x, y, width, height, title) if found, otherwise None.
    """
    # Specify a substring that uniquely identifies your window
    try:
        output = subprocess.check_output(["wmctrl", "-lG"]).decode("utf-8")
    except Exception as e:
        print("Failed to run wmctrl:", e)
        return None

    for line in output.splitlines():
        # wmctrl -lG output is expected to have 8 fields:
        # window_id, desktop, x, y, width, height, host, title
        parts = line.split(None, 7)
        if len(parts) < 8:
            continue
        win_id, desktop, x, y, width, height, host, title = parts
        if target_title in title:
            return int(x), int(y), int(width), int(height), title
    return None
def get_screenshot(x, y, width, height) -> np.ndarray:
    # --- Set up ctypes to call the C function ---
    # Load the shared library (ensure ss.so is in the current directory)
    lib = ctypes.CDLL("./ss.so")
    # Define the argument types for getScreen:
    # void getScreen(const int xx, const int yy, const int W, const int H, unsigned char *data)
    lib.getScreen.argtypes = [ctypes.c_int, ctypes.c_int, ctypes.c_int, ctypes.c_int,
                                ctypes.POINTER(ctypes.c_ubyte)]
    buf_size = width * height * 3
    buffer = (ctypes.c_ubyte * buf_size)()
    lib.getScreen(x, y, width, height, buffer)
    # Convert the C buffer into a NumPy array and reshape to (height, width, 3)
    img = np.frombuffer(buffer, dtype=np.uint8).reshape((height, width, 3))
    img = img[:, :, [2, 1, 0]]
    return img


In [3]:
title = "RMX2151" # Replace with your target window's title
geom = get_window_geometry_from_wmctrl(title) # Find the target window's geometry
if not geom:
    print("Window not found!")
    exit(1)
x, y, width, height, found_title = geom
print(f"Found window '{found_title}'")

# --- Main Loop: Capture and display the window screenshot ---
prev_time = time.time()
while True:
    img = get_screenshot(x, y, width, height) # Get the screenshot of the target window
    cv2.imshow("Computer Vision", img)
    fps = 1 / (time.time() - prev_time)
    prev_time = time.time()
    print(f"FPS: {fps:.2f}")
    if cv2.waitKey(1) == ord('q'):
        cv2.destroyAllWindows()
        break
print("done")

Found window 'RMX2151'
FPS: 5.01
FPS: 27.26
FPS: 32.20
FPS: 48.03
FPS: 64.10
FPS: 55.14
FPS: 31.82
FPS: 46.59
FPS: 42.62
FPS: 46.76
FPS: 36.46
FPS: 49.70
FPS: 45.17
FPS: 34.09
FPS: 39.19
FPS: 58.37
FPS: 61.85
FPS: 61.51
FPS: 64.94
FPS: 46.59
FPS: 64.34
FPS: 56.50
FPS: 54.92
FPS: 57.03
FPS: 54.03
FPS: 41.32
FPS: 44.49
FPS: 53.19
FPS: 47.47
FPS: 43.50
FPS: 39.19
FPS: 43.32
FPS: 57.41
FPS: 49.76
FPS: 52.95
FPS: 57.34
FPS: 39.74
FPS: 49.01
FPS: 60.47
FPS: 58.11
FPS: 60.59
FPS: 59.19
FPS: 52.81
FPS: 51.42
FPS: 56.15
FPS: 61.07
FPS: 58.24
FPS: 66.59
FPS: 63.81
FPS: 59.13
FPS: 39.25
FPS: 35.08
FPS: 48.40
FPS: 44.60
FPS: 40.08
FPS: 39.51
FPS: 77.82
FPS: 83.30
FPS: 62.55
FPS: 68.80
FPS: 77.72
FPS: 67.15
FPS: 65.97
FPS: 45.42
FPS: 69.35
FPS: 68.03
FPS: 69.68
FPS: 54.72
FPS: 54.36
FPS: 55.74
FPS: 42.27
FPS: 45.78
FPS: 67.74
FPS: 65.23
FPS: 68.32
FPS: 57.55
FPS: 40.22
FPS: 43.83
FPS: 36.10
FPS: 50.99
FPS: 70.75
FPS: 37.03
FPS: 58.15
FPS: 68.62
FPS: 72.66
FPS: 62.76
FPS: 77.03
FPS: 57.48
FPS: 65.97